In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import tensorflow as tf
import numpy as np
import pandas as pd

import itertools

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio"]
LABEL = "medv"

In [ ]:
def get_input_fn(dataset, num_epochs=None, shuffle=True):
    return tf.estimator.inputs.pandas_input_fn(
        x=pd.DataFrame({k: dataset[k].values for k in FEATURES}),
        y=pd.Series(dataset[LABEL].values),
        num_epochs=num_epochs,
        shuffle=shuffle)

In [ ]:
#Build the classifier
training_set = pd.read_csv("boston_train.csv", skipinitialspace=True, skiprows=1,names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True, skiprows=1,names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True, skiprows=1,names=COLUMNS)
print (training_set.size)
print (test_set.size)

feature_cols = [tf.feature_column.numeric_column(k) for k in FEATURES]
regressor = None
regressor = tf.estimator.DNNRegressor(
    feature_columns=feature_cols, 
    hidden_units=[10, 10], 
    model_dir="/tmp/boston3")

regressor.train(input_fn=get_input_fn(training_set), steps = 5000)

In [ ]:
#Evaluate the model and predict on new examples
evaluations = regressor.evaluate(input_fn=get_input_fn(test_set, num_epochs=1, shuffle=False))
loss = evaluations["loss"]
print ("Loss: {0:f}".format(loss))

predictions = list(regressor.predict(input_fn=get_input_fn(prediction_set, num_epochs=1, shuffle=False)))
pre = [p["predictions"] for p in predictions]
print("Predictions: {}".format(str(pre)))